# Azure AI Search s integrací NVIDIA NIM a LlamaIndex

V tomto notebooku si ukážeme, jak využít AI modely od NVIDIA a LlamaIndex k vytvoření výkonného pipeline pro Retrieval-Augmented Generation (RAG). Použijeme LLMs a embeddings od NVIDIA, integrujeme je s Azure AI Search jako vektorovým úložištěm a provedeme RAG pro zlepšení kvality a efektivity vyhledávání.

## Výhody
- **Škálovatelnost**: Využijte velké jazykové modely od NVIDIA a Azure AI Search pro škálovatelné a efektivní vyhledávání.
- **Úspora nákladů**: Optimalizujte vyhledávání a získávání dat pomocí efektivního vektorového úložiště a hybridních vyhledávacích technik.
- **Vysoký výkon**: Kombinujte výkonné LLMs s vektorovým vyhledáváním pro rychlejší a přesnější odpovědi.
- **Kvalita**: Udržujte vysokou kvalitu vyhledávání tím, že odpovědi LLM budou podloženy relevantními získanými dokumenty.

## Předpoklady
- 🐍 Python 3.9 nebo vyšší
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API klíč pro přístup k LLMs a embeddings od NVIDIA prostřednictvím mikroservisů NVIDIA NIM

## Pokryté funkce
- ✅ Integrace NVIDIA LLM (použijeme [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embeddings od NVIDIA (použijeme [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Pokročilé režimy získávání dat v Azure AI Search
- ✅ Indexování dokumentů pomocí LlamaIndex
- ✅ RAG pomocí Azure AI Search a LlamaIndex s LLMs od NVIDIA

Pojďme začít!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Instalace a požadavky
Vytvořte prostředí Python s použitím verze Pythonu >3.10.

## Začínáme!


Chcete-li začít, potřebujete `NVIDIA_API_KEY` pro použití modelů NVIDIA AI Foundation:  
1) Vytvořte si bezplatný účet na [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Klikněte na vámi zvolený model.  
3) V části Input vyberte záložku Python, klikněte na **Get API Key** a poté na **Generate Key**.  
4) Zkopírujte a uložte vygenerovaný klíč jako NVIDIA_API_KEY. Odtud byste měli mít přístup k endpointům.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Příklad RAG s použitím LLM a vkládání
### 1) Inicializace LLM
`llama-index-llms-nvidia`, známý také jako NVIDIA LLM konektor, umožňuje připojení a generování z kompatibilních modelů dostupných v katalogu API od NVIDIA. Zde najdete seznam modelů pro dokončování chatů: https://build.nvidia.com/search?term=Text-to-Text

Zde použijeme **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inicializace vkládání
`llama-index-embeddings-nvidia`, známý také jako konektor pro vkládání od NVIDIA, umožňuje připojení k kompatibilním modelům dostupným v katalogu API od NVIDIA a jejich generování. Vybrali jsme model `nvidia/nv-embedqa-e5-v5` jako model pro vkládání. Seznam modelů pro textové vkládání najdete zde: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Vytvořte Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Vytvořte dotazovací stroj pro kladení otázek nad vašimi daty

Zde je dotaz využívající čisté vektorové vyhledávání v Azure AI Search a ukotvující odpověď na náš LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Zde je dotaz využívající hybridní vyhledávání v Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analýza vyhledávání vektoru
Odpověď LLM přesně zachycuje klíčové ekonomické důsledky uvedené v zdrojovém textu týkající se ruského akciového trhu. Konkrétně uvádí, že ruský akciový trh zaznamenal výrazný pokles, ztratil 40 % své hodnoty a obchodování bylo pozastaveno kvůli probíhající situaci. Tato odpověď dobře odpovídá informacím uvedeným ve zdroji, což naznačuje, že LLM správně identifikoval a shrnul relevantní detaily týkající se dopadu na akciový trh v důsledku ruských kroků a uvalených sankcí.

#### Komentář ke zdrojovým uzlům
Zdrojové uzly poskytují podrobný popis ekonomických důsledků, kterým Rusko čelilo kvůli mezinárodním sankcím. Text zdůrazňuje, že ruský akciový trh ztratil 40 % své hodnoty a obchodování bylo pozastaveno. Dále zmiňuje další ekonomické dopady, jako je devalvace rublu a širší izolace ruské ekonomiky. Odpověď LLM účinně shrnula klíčové body z těchto uzlů, přičemž se zaměřila na dopad na akciový trh, jak bylo požadováno v dotazu.


Nyní se podívejme na dotaz, kde Hybridní vyhledávání neposkytuje dobře podloženou odpověď:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hybridní vyhledávání: Analýza odpovědi LLM
Odpověď LLM v příkladu hybridního vyhledávání naznačuje, že poskytnutý kontext neuvádí přesné datum invaze Ruska na Ukrajinu. Tato odpověď naznačuje, že LLM využívá informace dostupné ve zdrojových dokumentech, ale zároveň uznává absenci konkrétních detailů v textu.

Odpověď je přesná v tom, že identifikuje zmínky o událostech souvisejících s ruskou agresí, ale neuvádí konkrétní datum invaze. To ukazuje schopnost LLM porozumět poskytnutým informacím a zároveň rozpoznat mezery v obsahu. LLM účinně vybízí uživatele, aby hledal externí zdroje nebo historické záznamy pro přesné datum, což demonstruje určitou opatrnost při práci s neúplnými informacemi.

### Analýza zdrojových uzlů
Zdrojové uzly v příkladu hybridního vyhledávání obsahují výňatky z projevu, který se zabývá reakcí USA na ruské akce na Ukrajině. Tyto uzly zdůrazňují širší geopolitický dopad a kroky, které USA a jejich spojenci podnikli v reakci na invazi, ale nezmiňují konkrétní datum invaze. To odpovídá odpovědi LLM, která správně identifikuje, že kontext postrádá přesné informace o datu.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: Analýza odpovědi LLM
V příkladu Hybrid w/Reranking poskytuje odpověď LLM další kontext tím, že uvádí, že událost nastala šest dní před tím, než byl projev přednesen. To naznačuje, že LLM dokáže odvodit datum invaze na základě načasování projevu, i když stále potřebuje znát přesné datum projevu pro dosažení přesnosti.

Tato odpověď ukazuje zlepšenou schopnost využívat kontextové vodítka k poskytnutí informativnější odpovědi. Zdůrazňuje výhodu rerankingu, kdy LLM může přistupovat k relevantnějším informacím a upřednostňovat je, aby poskytlo přesnější přiblížení požadovaného detailu (tj. data invaze).

### Analýza zdrojových uzlů
Zdrojové uzly v tomto příkladu zahrnují odkazy na načasování ruské invaze, konkrétně zmiňují, že k ní došlo šest dní před projevem. Přestože přesné datum není stále explicitně uvedeno, uzly poskytují časový kontext, který umožňuje LLM poskytnout nuancovanější odpověď. Zahrnutí tohoto detailu ukazuje, jak reranking může zlepšit schopnost LLM extrahovat a odvozovat informace z poskytnutého kontextu, což vede k přesnější a informativnější odpovědi.


**Poznámka:**  
V tomto notebooku jsme použili mikroservisy NVIDIA NIM z katalogu API NVIDIA.  
Výše uvedená API, `NVIDIA (llms)`, `NVIDIAEmbedding` a [Azure AI Search Semantic Hybrid Retrieval (vestavěné přehodnocování)](https://learn.microsoft.com/azure/search/semantic-search-overview). Upozorňujeme, že výše uvedená API mohou také podporovat mikroservisy hostované na vlastním serveru.  

**Příklad:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")
```



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o co největší přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho původním jazyce. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
